<a href="https://colab.research.google.com/github/liamti5/Essentials-in-Text-and-Speech-Processing/blob/master/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=23b8170a4ea6c67e8ddb22121238fa381698e5d8e9fa0a5f750ed98345803ca7
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import sentencepiece

In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    print("using GPU")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

using GPU


In [38]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch.cuda.device(0)
cuda0 = torch.device('cuda:0')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")
classifier = pipeline('translation', model=model, tokenizer=tokenizer, device=device)

res = classifier('I love it when I have to work late....  just kidding')

print(res)

[{'translation_text': 'Ich liebe es, wenn ich spät arbeiten muss...'}]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
with open('/content/drive/My Drive/Colab/Uni/HS23/EITASP/reviews_en.csv', 'r') as f:
    df = pd.read_csv(f)

df.head()

,_id,review,score,university,course,upvotes,downvotes,date,courseName,courseNameShort
0,5a5e48e1669270001a255a05,HSG too good for this,1,UZH,50044544,4.0,41.0,1548419537136,Corporate Finance (L+E),Corporate Finance
1,5a5e4c87669270001a255a9e,Winkelmann = Ehrenmann,5,UZH,50047387,1.0,0.0,1608566977197,Einführung in die empirische Wirtschaftsforsch...,Einführung in die empirische Wirtschaftsforschung
2,5a5e50f3be6796001a9d7316,PROs: This module represents for the student t...,3,UZH,50772388,3.0,0.0,1578223557075,CHE 212 Practical course in Synthetic Chemistry,Practical course in Synthetic Chemistry
3,5a5e5127be6796001a9d7324,Great Prof.! Best one so far. Easy to get good...,5,UZH,50772226,0.0,1.0,1632840742122,CHE 203 Organic Chemistry I,Organic Chemistry I
4,5a5e5161be6796001a9d7337,The AC part is great but OC is trying to pack ...,2,UZH,50740660,0.0,1.0,1626375246178,"CHE 102 Fundamentals of Chemistry, Part 2","Fundamentals of Chemistry, Part 2"


In [6]:
# Process translations with batch processing and error handling
translations = []

len_df = len(df)
for index, row in df.iterrows():
    print(len_df)
    review = row["review"]
    print(review)
    if len(review) <= 512:
      try:
          translation = classifier(review, max_length=512)[0]["translation_text"]
          print(translation)
          translations.append(translation)
      except Exception as e:
          print(e)
          translations.append("")
      finally:
          len_df -= 1
    else:
      translations.append("")

3083
HSG too good for this
HSG zu gut dafür
3082
Winkelmann = Ehrenmann
Winkelmann = Ehrenmann
3081
PROs: This module represents for the student the first chance to face independent synthetical work from A to Z. You learn (or often, you self-learn) how to do literature research, perform syntheses under exclusion of air/water/light, purify the substances by means of different methods and get the chance to perform a full characterisation of the synthesized product. It is therefore a module which really teaches you a lot. The majority of the teaching assistants involved took their job responsibly and invested a lot of time to correct the laboratory reports properly (and there are MANY criteria of corrections set for this course). CONs: this module is always overbooked. There aren’t enough places for everybody, so there’s a rotating place system with a computational week course to periodically reduce the number of people present in the lab. It’s a shame that the university is making new ch

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Streaming output truncated to the last 5000 lines.
1789
Bad TA, mathematica is useless, assignments unclear, questions not answered, many mistakes in the material
Bad TA, Mathematica ist nutzlos, Aufgaben unklar, Fragen nicht beantwortet, viele Fehler im Material
1788
Florian and Owen are amazing instructors. The course was a lot of fun. The assistants Jeanine and Roman as well as the others from eawag who came to help were very knowledgable and great support. It would be nice to have spent a little more time up in Elm though!
Florian und Owen sind tolle Instruktoren. Der Kurs war eine Menge Spaß. Die Assistenten Jeanine und Roman sowie die anderen aus Eawag, die kamen, um zu helfen, waren sehr knowledgable und große Unterstützung. Es wäre schön, ein wenig mehr Zeit in Elm verbracht haben, obwohl!
1787
Anna-Liisa and all her post docs are great lecturers. The topics were easy, but the approaches novel.
Anna-Liisa und alle ihre Postdocs sind große Dozenten. Die Themen waren einfach, abe

In [7]:
print(translations)

['HSG zu gut dafür', 'Winkelmann = Ehrenmann', '', 'Große Prof.! Beste bis jetzt. Einfach gute Noten zu bekommen, wenn Sie alle Übungen, die er vorschlägt. Einfach genug üben.', 'Der AC-Teil ist toll, aber OC versucht, zu viel Zeug in zu wenig Zeit zu packen. Die Prüfung war nichts wie die Übungen und zu hart, wenn man bedenkt, die Chancen, die Schüler vorbereiten mussten.', 'würde nicht empfehlen', 'mit einem Gehalt an Milchfett von mehr als 30 GHT, bezogen auf die Trockenmasse', 'Super interessante Vorlesung. Leider war der Professor, der die Paläontologie Teil lehrte extrem unorganisiert. Zeitmanagement wa', 'Sehr interessant. Und nein, die Prüfung enthielt keine zu detaillierten Fragen. Ich hatte einen Blick auf die Dias am Abend zuvor und bekam eine 5. Wenn Sie während der Vorlesung genau zuhören, müssen Sie nicht viel Zeit in das Studium investieren.', 'Best Professor, sie versucht ihr Bestes, damit Sie den Kurs verstehen, sehr verantwortlich, sehr interaktiv. Nie bereuen, ihren 

In [12]:
df.insert(loc = 2,
          column = 'translations',
          value = translations)

ValueError: ignored

In [14]:
df.to_csv('/content/drive/My Drive/Colab/Uni/HS23/EITASP/reviews_en_translated.csv', index=False)